In [1]:
import torch
import yaml
import ast
#from dig.threedgraph.dataset import QM93D
from dig.threedgraph.dataset import MD17
from dig.threedgraph.dataset.PygTobermorite import Tobermorite
from dig.threedgraph.method import SphereNet #SchNet, DimeNetPP, ComENet
from dig.threedgraph.method import run
from dig.threedgraph.evaluation import ThreeDEvaluator

# Load configs

In [2]:
with open('config.yaml', 'r') as c:
    config = yaml.safe_load(c)
    
# For strings that yaml doesn't parse (e.g. None)
for key, val in config.items():
    if type(val) is str:
        try:
            config[key] = ast.literal_eval(val)
        except (ValueError, SyntaxError):
            pass

In [3]:
name = config['name']
n_train = config['n_train']
n_val = config['n_val']
seed = config['seed']
energy_and_force = config['energy_and_force']
cutoff = config['cutoff']
num_layers = config['num_layers']
hidden_channels = config['hidden_channels']
out_channels = config['out_channels']
int_emb_size = config['int_emb_size']
basis_emb_size_dist = config['basis_emb_size_dist']
basis_emb_size_angle = config['basis_emb_size_angle']
basis_emb_size_torsion = config['basis_emb_size_torsion']
out_emb_channels = config['out_emb_channels']
num_spherical = config['num_spherical']
num_radial = config['num_radial']
envelope_exponent = config['envelope_exponent']
num_before_skip = config['num_before_skip']
num_after_skip = config['num_after_skip']
num_output_layers = config['num_output_layers']

epochs = config['epochs']
batch_size = config['batch_size']
vt_batch_size = config['vt_batch_size']
lr = config['lr']
lr_decay_factor = config['lr_decay_factor']
lr_decay_step_size = config['lr_decay_step_size']

In [10]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda', index=0)

# Dataset

In [5]:
dataset = Tobermorite(root='dataset/', name=name)

split_idx = dataset.get_idx_split(len(dataset.data.y), train_size=n_train, valid_size=n_val, seed=seed)

train_dataset, valid_dataset, test_dataset = dataset[split_idx['train']], dataset[split_idx['valid']], dataset[split_idx['test']]
print('train, validaion, test:', len(train_dataset), len(valid_dataset), len(test_dataset))

train, validaion, test: 950 50 6000


/home/zkm/anaconda3/envs/spherenet/lib/python3.10/site-packages/torch_geometric/data/in_memory_dataset.py:300: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


# Model

In [6]:
model = SphereNet(energy_and_force=energy_and_force, cutoff=cutoff, num_layers=num_layers, 
        hidden_channels=hidden_channels, out_channels=out_channels, int_emb_size=int_emb_size, 
        basis_emb_size_dist=basis_emb_size_dist, basis_emb_size_angle=basis_emb_size_angle, 
        basis_emb_size_torsion=basis_emb_size_torsion, out_emb_channels=out_emb_channels, 
        num_spherical=num_spherical, num_radial=num_radial, envelope_exponent=envelope_exponent, 
        num_before_skip=num_before_skip, num_after_skip=num_after_skip, num_output_layers=num_output_layers 
        )
loss_func = torch.nn.L1Loss()
evaluation = ThreeDEvaluator()

# FLOPs

In [17]:
from torch_geometric.data import DataLoader
from tqdm import tqdm
from thop import profile

train_loader = DataLoader(train_dataset, batch_size, shuffle=True)
for step, batch_data in enumerate(tqdm(train_loader)):
    print(device)
    batch_data = batch_data.to(device)
    inputs = batch_data
    model = model.to(device)
    flops,params=profile(model,(inputs, ))
    print(flops,params)

  0%|                                                                                                                                             | 0/238 [00:00<?, ?it/s]

cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.


  1%|█▋                                                                                                                                   | 3/238 [00:00<00:55,  4.23it/s]

30319612288.0 1877952.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
30383301760.0 1877952.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
31390812544.0 1877952.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
30620603776.0 1877952.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.


  3%|███▉                                                                                                                                 | 7/238 [00:01<00:24,  9.42it/s]

...

30666563200.0 1877952.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
30400952960.0 1877952.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
30827942656.0 1877952.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
30896955264.0 1877952.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍| 237/238 [00:13<00:00, 18.09it/s]

30352442624.0 1877952.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
29014520192.0 1877952.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
30612455552.0 1877952.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
30208176768.0 1877952.0
cuda:0
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 238/238 [00:13<00:00, 17.02it/s]

15497767936.0 1877952.0
